Data Collection with CCXT

In [ ]:
import ccxt
import pandas as pd

import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# Initialize the exchange
exchange = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '1d'
since = exchange.parse8601('2021-01-01T00:00:00Z')
now = exchange.milliseconds()

all_candles = []

while since < now:
    candles = exchange.fetch_ohlcv(symbol, timeframe, since)
    if len(candles) == 0:
        break
    since = candles[-1][0] + 1  # start the next fetch from the end of the last one
    all_candles += candles

# Convert to DataFrame
df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

# Create a copy with the closing price for forecasting
data = df[['date', 'close', 'volume']].copy()
data.rename(columns={'date': 'ds', 'close': 'y', 'volume': 'volume'}, inplace=True)

print(data.head())


Plot the historical Bitcoin prices

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(data['ds'], data['y'])
plt.title('Historical Bitcoin Price from Binance')
plt.xlabel('Date')
plt.ylabel('Price (in USDT)')
plt.grid(True)
plt.show()

Add additional regressors/features & Clean Data

In [ ]:
import talib

data['rsi'] = talib.RSI(data['y'].values, timeperiod=16)
data.dropna(subset=['rsi'], inplace=True)

data['obv'] = talib.OBV(data['y'].values, data['volume'].values)
data.dropna(subset=['obv'], inplace=True)

Fit and Train the Prophet model with features/regressors

In [ ]:
from prophet import Prophet

# Initialize the Prophet model
model = Prophet()

# Add RSI as a regressor
model.add_regressor('rsi')

# Add OBV as a regressor
model.add_regressor('obv')

# Train the model with the data
model.fit(data)

Predict the price for the next day

In [ ]:
# Predict Bitcoin price for the next day
future_day = model.make_future_dataframe(periods=1, freq='D', include_history=False)

# Use the most recent RSI and OBV values for future predictions
last_rsi_value = data['rsi'].iloc[-1]
last_obv_value = data['obv'].iloc[-1]

future_day['rsi'] = last_rsi_value
future_day['obv'] = last_obv_value

forecast_day = model.predict(future_day)
next_day_price = forecast_day['yhat'].values[0]
print(f"Predicted Bitcoin price for the next day: ${next_day_price:.2f}")

Create future date data for which we want to predict the Bitcoin prices and then generate forecasts. Also demonstrate Forecasting with the Regressor/Feature

In [ ]:
# Create future dates for prediction (e.g., for the next 365 days)
future = model.make_future_dataframe(periods=365)

# Use the most recent RSI and OBV values for future predictions
last_rsi_value = data['rsi'].iloc[-1]
last_obv_value = data['obv'].iloc[-1]

future['rsi'] = last_rsi_value
future['obv'] = last_obv_value

# Generate forecasts
forecast = model.predict(future)

Visualize the predicted prices

In [ ]:
fig1 = model.plot(forecast)

 Break down the forecast into its various components

In [ ]:
fig2 = model.plot_components(forecast)

Setup & Perform Cross-Validation

In [ ]:
from prophet.diagnostics import cross_validation

# Here's an example setup:
# Initial training period = 730 days (2 years)
# Horizon = 90 days
# Period = 30 days (i.e., make a forecast every 30 days)

df_cv = cross_validation(model, initial='730 days', period='30 days', horizon='90 days')


Compute Performance Metrics

In [ ]:
from prophet.diagnostics import performance_metrics

df_p = performance_metrics(df_cv)
print(df_p.head())

Visualizing Performance

In [ ]:
from prophet.plot import plot_cross_validation_metric

fig = plot_cross_validation_metric(df_cv, metric='mape')

Compute the percentage accuracy from MAPE

In [ ]:
# Assuming df_p is the result from the performance_metrics function
mape = df_p['mape'].mean()
accuracy = 100 - (mape * 100)
print(f"Forecast Accuracy: {accuracy:.2f}%")